
##Content-Based Recommendation System








In [9]:
# +-------------------------------+
# |           Start               |
# +-------------------------------+
#               |
#               v
# +-------------------------------+
# | Load datasets: movies.csv &   |
# | credits.csv                   |
# +-------------------------------+
#               |
#               v
# +-------------------------------+
# | Merge datasets on 'title'     |
# +-------------------------------+
#               |
#               v
# +-------------------------------+
# | Select relevant columns:       |
# | movie_id, title, overview,    |
# | genres, keywords, cast, crew  |
# +-------------------------------+
#               |
#               v
# +-------------------------------+
# | Preprocess data:               |
# | - Convert JSON strings to     |
# |   lists                       |
# | - Keep top 3 cast members     |
# | - Extract director            |
# | - Remove spaces from names    |
# | - Tokenize overview text      |
# +-------------------------------+
#               |
#               v
# +-------------------------------+
# | Combine text fields into a     |
# | single 'tags' column           |
# +-------------------------------+
#               |
#               v
# +-------------------------------+
# | Vectorize 'tags' using         |
# | CountVectorizer (max 5000 features) |
# +-------------------------------+
#               |
#               v
# +-------------------------------+
# | Compute cosine similarity      |
# | matrix for all movies          |
# +-------------------------------+
#               |
#               v
# +-------------------------------+
# | Define recommendation function |
# | - Find movie index             |
# | - Retrieve similarity scores   |
# | - Sort and select top 5 movies |
# +-------------------------------+
#               |
#               v
# +-------------------------------+
# | Test recommendation function   |
# +-------------------------------+
#               |
#               v
# +-------------------------------+
# | Save movie list and similarity |
# | matrix using pickle            |
# +-------------------------------+
#               |
#               v
# +-------------------------------+
# |             End                |
# +-------------------------------+


In [10]:
# from google.colab import drive
# drive.mount('/content/drive')

## Importing required libraries

In [11]:

import numpy as np  # for numerical operations
import pandas as pd  # for data manipulation and analysis (mainly using DataFrames)
import os  # for interacting with the operating system (e.g., reading files)


## Reading the movies and credits data

In [12]:

movies = pd.read_csv('data/tmdb_5000_movies.csv')
credits = pd.read_csv('data/tmdb_5000_credits.csv')



## Display first 2 rows of the movies dataframe

In [13]:

movies.head(2)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


## Checking shape (number of rows and columns) of movies data

In [14]:

movies.shape



(4803, 20)

## Previewing the credits dataset

In [15]:

credits.head()


,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


## Merging both datasets on the "title" column to get a single dataset


In [16]:
movies = movies.merge(credits, on='title')


## Displaying the combined dataset

In [17]:

movies.head()


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


## Selecting only the required columns for the recommendation system


In [18]:
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
movies.head()


,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


## Importing Abstract Syntax Trees (for safely evaluating strings that look like lists/dictionaries)


In [19]:
import ast



## Function to extract only the 'name' field from a stringified list of dictionaries (e.g., genres, keywords)


In [20]:
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name'])
    return L


## Dropping rows with missing (NaN) values to clean the data

In [21]:

movies.dropna(inplace=True)



## Applying the convert function to 'genres' column

In [22]:


movies['genres'] = movies['genres'].apply(convert)
movies.head()



,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


## Applying the same function to 'keywords' column


In [23]:
movies['keywords'] = movies['keywords'].apply(convert)
movies.head()




,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


## Function to get top 3 cast members from the list


In [24]:
def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter += 1
    return L



## Extracting top 3 cast members


In [25]:
movies['cast'] = movies['cast'].apply(convert)
movies['cast'] = movies['cast'].apply(lambda x: x[0:3])




## Function to extract the director from the crew list


In [26]:
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L


## Applying the function to get directors


In [27]:
movies['crew'] = movies['crew'].apply(fetch_director)



## Sample records to see how the data looks


In [28]:
movies.sample(15)



,movie_id,title,overview,genres,keywords,cast,crew
889,8741,The Thin Red Line,"Based on the graphic novel by James Jones, The...","[Drama, History, War]","[based on novel, japanese, world war ii, battl...","[Sean Penn, Adrien Brody, Jim Caviezel]",[Terrence Malick]
2332,60307,Diary of a Wimpy Kid: Rodrick Rules,"Back in middle school after summer vacation, G...","[Family, Comedy]","[based on novel, middle school]","[Zachary Gordon, Devon Bostick, Rachael Harris]",[David Bowers]
2239,256962,Little Boy,An eight-year-old boy is willing to do whateve...,"[Comedy, Drama, War]","[japanese, world war ii, spirituality]","[Jakob Salvati, Emily Watson, Cary-Hiroyuki Ta...",[Alejandro Monteverde]
3259,13751,Akeelah and the Bee,Eleven year-old Akeelah Anderson's life is not...,[Drama],"[black people, spelling, spelling bee]","[Keke Palmer, Laurence Fishburne, Angela Bassett]",[Doug Atchison]
4017,9591,That Thing You Do!,A Pennsylvania band scores a hit in 1964 and r...,"[Comedy, Drama]","[pop star, musical, record label, music, music...","[Tom Everett Scott, Johnathon Schaech, Tom Hanks]",[Tom Hanks]
3937,13982,8 Heads in a Duffel Bag,Mafia hitman Tommy Spinelli (Joe Pesci) is fly...,"[Comedy, Crime]","[mexico, vacation, murder, head]","[Joe Pesci, Kristy Swanson, George Hamilton]",[Tom Schulman]
4769,125537,Smiling Fish & Goat On Fire,Two brothers share a house in LA's Fairfax dis...,"[Comedy, Romance]",[],"[Derick Martini, Amy Hathaway, Steven Martini]",[Kevin Jordan]
1606,13156,Secondhand Lions,"""Secondhand Lions"" follows the comedic adventu...","[Comedy, Drama, Family]","[texas, money, veteran]","[Michael Caine, Robert Duvall, Haley Joel Osment]",[Tim McCanlies]
3250,40807,50/50,"Inspired by a true story, a comedy centered on...","[Comedy, Drama]","[painter, father son relationship, therapist, ...","[Joseph Gordon-Levitt, Seth Rogen, Anna Kendrick]",[Jonathan Levine]
2721,23988,Stan Helsing,It's Halloween night and video store clerk Sta...,"[Comedy, Horror]","[halloween, slapstick, duringcreditsstinger]","[Diora Baird, Leslie Nielsen, Steve Howey]",[Bo Zenga]


## Function to remove spaces in multi-word names (e.g., "Tom Cruise" → "TomCruise")


In [29]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ", ""))
    return L1



## Removing spaces in names to treat them as single tokens


In [30]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)


In [31]:
movies['genres'][0]

['Action', 'Adventure', 'Fantasy', 'ScienceFiction']

In [32]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


## Splitting the overview text into individual words


In [33]:

movies['overview'] = movies['overview'].apply(lambda x: x.split())




In [34]:
movies['overview'][0]

['In',
 'the',
 '22nd',
 'century,',
 'a',
 'paraplegic',
 'Marine',
 'is',
 'dispatched',
 'to',
 'the',
 'moon',
 'Pandora',
 'on',
 'a',
 'unique',
 'mission,',
 'but',
 'becomes',
 'torn',
 'between',
 'following',
 'orders',
 'and',
 'protecting',
 'an',
 'alien',
 'civilization.']

In [35]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


## Creating a new column 'tags' that combines overview, genres, keywords, cast, and crew


In [36]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']



In [37]:
movies['tags'][0]

['In',
 'the',
 '22nd',
 'century,',
 'a',
 'paraplegic',
 'Marine',
 'is',
 'dispatched',
 'to',
 'the',
 'moon',
 'Pandora',
 'on',
 'a',
 'unique',
 'mission,',
 'but',
 'becomes',
 'torn',
 'between',
 'following',
 'orders',
 'and',
 'protecting',
 'an',
 'alien',
 'civilization.',
 'Action',
 'Adventure',
 'Fantasy',
 'ScienceFiction',
 'cultureclash',
 'future',
 'spacewar',
 'spacecolony',
 'society',
 'spacetravel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alienplanet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'loveaffair',
 'antiwar',
 'powerrelations',
 'mindandsoul',
 '3d',
 'SamWorthington',
 'ZoeSaldana',
 'SigourneyWeaver',
 'JamesCameron']

## Dropping the original columns that were combined into 'tags'


In [38]:
new = movies.drop(columns=['overview', 'genres', 'keywords', 'cast', 'crew'])


## Joining the list of words in 'tags' into a single string (space-separated)


In [39]:
new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new.head()


,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [40]:
new['tags']

0       In the 22nd century, a paraplegic Marine is di...
1       Captain Barbossa, long believed to be dead, ha...
2       A cryptic message from Bond’s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a war-weary, former military ca...
                              ...                        
4804    El Mariachi just wants to play his guitar and ...
4805    A newlywed couple's honeymoon is upended by th...
4806    "Signed, Sealed, Delivered" introduces a dedic...
4807    When ambitious New York attorney Sam is sent t...
4808    Ever since the second grade when he first saw ...
Name: tags, Length: 4806, dtype: object

## Converting text data into numerical vectors using Bag of Words model (with max 5000 features and removing English stopwords)


In [41]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')
vector = cv.fit_transform(new['tags']).toarray()



In [42]:
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

## Checking shape of the resulting matrix (rows = number of movies, columns = vocabulary size)


In [43]:
vector.shape


(4806, 5000)

## Calculating cosine similarity between all movie vectors


In [44]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)



## Sample similarity values


In [45]:
similarity[0]

array([1.        , 0.08964215, 0.06071767, ..., 0.02519763, 0.0277885 ,
       0.        ])

* Because cosine_similarity computes similarity between every pair of rows (movies) in vector.

* Each row corresponds to one movie’s feature vector.

* So you get similarity scores for all pairs of 4806 movies.

In [46]:
similarity.shape


(4806, 4806)

## Helper function to recommend similar movies based on cosine similarity


In [47]:

def recommend(movie):
    # Find index of the given movie
    index = new[new['title'] == movie].index[0]
    # Get similarity scores for that movie with all other movies
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
    # Print top 5 most similar movies (excluding the input movie itself)
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)



##This is how recommend function works:

###Step-by-step explanation of the **`recommend`** function

---

### Purpose:

This function **finds and prints movies similar to the given movie** based on their content similarity (using cosine similarity scores).

---

### Breakdown of the code:

```python
def recommend(movie):
```

* This defines a function named `recommend` that takes one input parameter, `movie` — the movie title you want recommendations for.

---

```python
index = new[new['title'] == movie].index[0]
```

* This line finds the **row index** of the movie in the dataset `new` where the title matches the given `movie` name.
* `new['title'] == movie` creates a boolean filter.
* `.index[0]` extracts the first matching index.
* This index will be used to look up similarity scores for that movie.

---

```python
distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
```

* `similarity` is a matrix where each row corresponds to a movie, and each column contains similarity scores between that movie and every other movie.
* `similarity[index]` gives the similarity scores of the input movie with all other movies.
* `enumerate()` pairs each similarity score with its movie index, creating tuples like `(movie_index, similarity_score)`.
* `list()` converts the enumerate object into a list of tuples.
* `sorted(..., reverse=True, key=lambda x: x[1])` sorts this list **in descending order** by the similarity score (`x[1]`), so the most similar movies come first.

---

```python
for i in distances[1:6]:
    print(new.iloc[i[0]].title)
```

* `distances[1:6]` slices the sorted list to get the **top 5 movies** after the first one.
* We skip the first element `distances[0]` because that is the movie itself (with similarity = 1).
* The loop goes through these 5 most similar movies.
* `new.iloc[i[0]].title` fetches the title of the movie using the index `i[0]` from the original dataframe.
* `print()` displays the recommended movie titles.

---

### Summary:

* You give the function a movie name.
* It finds the similarity scores of that movie with all others.
* It sorts the movies by similarity score, highest first.
* It prints the top 5 similar movie titles (excluding the movie you searched for).


## Example usage of the recommendation system


In [48]:
recommend('Terminator 3: Rise of the Machines')



The Terminator
Terminator Salvation
Terminator 2: Judgment Day
Terminator Genisys
Highlander: Endgame


## Saving the data and similarity matrix to files so that they can be used in a web app or deployed system


In [49]:
import pickle
pickle.dump(new, open('movie_list.pkl', 'wb'))
pickle.dump(similarity, open('similarity.pkl', 'wb'))


## Explaination for Recommend function

In [50]:
new[new['title'] == "X-Men: Days of Future Past"].index[0]

46

In [51]:
list(enumerate(similarity[46]))

[(0, 0.1765469659009499),
 (1, 0.12309149097933274),
 (2, 0.1250610798961445),
 (3, 0.02051524849655546),
 (4, 0.16116459280507608),
 (5, 0.09750298988361741),
 (6, 0.023911404992940522),
 (7, 0.23354968324845696),
 (8, 0.09869275424396534),
 (9, 0.13273296830047474),
 (10, 0.24242424242424246),
 (11, 0.08362420100070908),
 (12, 0.1315903389919538),
 (13, 0.04875149494180871),
 (14, 0.2059714602177749),
 (15, 0.052486388108147805),
 (16, 0.16943474841747155),
 (17, 0.19897095972842185),
 (18, 0.1844571903642866),
 (19, 0.1348399724926484),
 (20, 0.07987230638308718),
 (21, 0.0646508183835236),
 (22, 0.07106690545187017),
 (23, 0.10050378152592121),
 (24, 0.05802588531856595),
 (25, 0.019968076595771794),
 (26, 0.29424494316824984),
 (27, 0.12070113739631691),
 (28, 0.0969762275752854),
 (29, 0.04875149494180871),
 (30, 0.10050378152592122),
 (31, 0.1973855084879307),
 (32, 0.07041787902195304),
 (33, 0.39927470475234533),
 (34, 0.03553345272593508),
 (35, 0.1519474352795173),
 (36, 0.1

In [52]:
sorted(list(enumerate(similarity[46])), reverse=True, key=lambda x: x[1])[0:5]

[(46, 1.0000000000000002),
 (33, 0.39927470475234533),
 (203, 0.36363636363636376),
 (64, 0.3555795011093798),
 (101, 0.3333333333333334)]